# Boulder Precinct Choropleth Map ETL
    
Create data files required for a choropleth map showing values by Boulder precincts.
- Transform the input data to GeoJson files compatible with Folium and Leaflet maps
- Save precinct geometry GeoJson files that  Folium Choropleth() can read as geometry
    - GeoJson Feature Collection with Feature id=precinct_name
- Started: August 10, 2022
- Updated: December 16, 2024

## Input Files

- Boulder County Precincts
    - Shape files downloaded from Boulder County on December 15, 2024
    - https://opendata-bouldercounty.hub.arcgis.com/datasets/c8e2897d283b47f780920af0827d5126_0/explore?location=40.086460%2C-105.373100%2C10.51

## Output Files

GeoJson files for Folium Choropleth maps
- Boulder Precinct geometry

## Issues
- Folium.Choropleth() is very picky about the data in the geometry file
    - needs feature_id=precinct_name to merge choropleth data layers


In [1]:
import pandas as pd
import geopandas as gpd
import folium

### Precincts - Boulder County Precinct Boundaries

In [ ]:
pctgdf = gpd.read_file('data/pct/Precincts.shp', columns=['PRECINCT','geometry'])
pctgdf = pctgdf.rename(columns={'PRECINCT':'precinct'})
pctgdf['precinct'] = pctgdf['precinct'].astype('int64')

# convert crs to WGS 84, ESPG:4326
pctgdf.to_crs(4326, inplace=True)
pctgdf.info()

In [ ]:
pctgdf.head(4)

In [ ]:
pctjson = pctgdf.to_json(drop_id=True)
# print(pctjson)
pctjson

In [ ]:
# get election votes results data
pres = pd.read_csv('data/election_results/election_results_2024_president_boulder.csv')
pres.info()

In [ ]:
# get turnout data
turnout = pd.read_csv('data/election_results/Vote Data Results Active Only(Precinct).csv', dtype=str)
turnout = turnout[['Precinct - Full', 'Target 92+ Voted %']]
turnout.info()
turnout.head(4)

##  PREFERRED METHOD TO CREATE FOLIUM CHOROPLETH LAYER

### Create choropleth layer with gdf for geometry and df for data
```
folium.Choropleth(
    geo_data = <geodataframe>,
    key_on = 'feature.properties.<geodata_key>', # defines key column in 'geo_data'
    data = <dataframe>,
    columns = ['<dataframe_key>', '<dataframe_column>'], # columns[0], dataframe_key, defines key column in 'data'
    fill_color = "YlGn",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    name = '<layer_name>',
    legend_name = '<legend_name>'
)

```

In [ ]:
#########   PREFERRED METHOD GeoDataFrame + DataFrame   #####################
# Create Choropleth layer showing Dem votes for President
# use GeoDataFrame for geometry
# use pres dataframe for data
# NOTE: preliminary voting results

folium.Choropleth(
    geo_data=pctgdf,
    key_on="feature.properties.precinct",
    data=pres,
    columns=["precinct_name", "dem_votes_pres"],
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dem Votes for President",
    name="Presidential Dem Votes",
).add_to(m)

folium.LayerControl().add_to(m)

# Display the Choropleth
m

In [ ]:
# Create Choropleth layer showing Dem Margin for President
# NOTE: preliminary voting results

folium.Choropleth(
    geo_data=pctjson,
    name="Presidential Dem Margin",
    data=pres,
    columns=["precinct_name", "dem_margin_pres"],
    key_on="feature.properties.precinct",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dem Margin for President",
).add_to(m)

folium.LayerControl().add_to(m)

# Display the Choropleth
m

In [ ]:
##################   ALTERNATIVE METHOD THAT MIMICS EXAMPLES IN FOLIUM DOCUMENTATION  ##########
# use GeoJson string for geometry
# use pres dataframe for data
# NOTE: preliminary voting results

# Create basemap layer that shows Boulder County
m = folium.Map(tiles='cartodb positron', location=(40.1, -105.3), zoom_level=10)

# Create Choropleth layer showing Dem votes for President
folium.Choropleth(
    geo_data=pctjson,
    name="Presidential Dem Votes",
    data=pres,
    columns=["precinct_name", "dem_votes_pres"],
    key_on="feature.properties.precinct",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dem Votes for President",
).add_to(m)

folium.LayerControl().add_to(m)

# Display the Choropleth
m

########  Try with final data from Geoff  ###########

Needs a lot of data cleaning

In [ ]:
# Create Target 92+ Turnout% choropleth map from Geoff's data
# NOTE:  needs data cleaning
#           % strings to numeric
#           Precinct - Full to int64

mturnout = folium.Map(tiles='cartodb positron', location=(40.1, -105.3), zoom_level=10)

# Create Choropleth layer showing Dem votes for President


folium.Choropleth(
    geo_data=pctgdf,
    name="Target 92+ Turnout%",
    data=turnout,
    columns=["Precinct - Full", "Target 92+ Voted %"],
    key_on="feature.properties.precinct",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Target 92+ Turnout",
).add_to(mturnout)

folium.LayerControl().add_to(mturnout)

# Display the Choropleth
mturnout